In [ ]:
x = [0.25, 0.5, 0.75]

y = data_3_nodups[data_3_nodups['DRAW'].isnull()].loc[:, ['TO_4_25', 'TO_4_5', 'TO_4_75',
                                                          'TU_4_25', 'TU_4_5', 'TU_4_75',]]
y['_25'] = fair_odd(y['TO_4_25'], margin_2(y['TO_4_25'], y['TU_4_25']))
y['_5'] = fair_odd(y['TO_4_5'], margin_2(y['TO_4_5'], y['TU_4_5']))
y['_75'] = fair_odd(y['TO_4_75'], margin_2(y['TO_4_75'], y['TU_4_75']))

np.polyfit(x, y.iloc[5, -3:], 5)